Import all required packages for this to run. 

In [1]:
import os.path
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import argparse
import pdb_parser
import pandas as pd
import glob
from scipy.spatial import KDTree
from sklearn.cluster import KMeans
import math
import logomaker
import numpy as np
import helper_functions
import sequence_logo_main
%matplotlib notebook

In [8]:

target_chain = input("Type the chain identifier for your target: ")
binder_chain = input("Type the chain identifier for your binders: ")
is_ligand = input("Is your target a ligand? [Y/N]: ")

if is_ligand == "Y":
    is_ligand = True
else:
    is_ligand = False
    res_str = input("Type the residues in the target you'd like to visualize separated by a comma (ex: 1,232,3123,2,5,7), or type all to visualize all residues: ")
    
if not res_str == all 

Type the chain identifier for your target: A
Type the chain identifier for your binders: B
Is your target a ligand? [Y/N]: N
Type the residues in the target you'd like to visualize separated by a comma (ex: 1,232,3123,2,5,7), or type all to visualize all residues: all


In [3]:

sequence_logo_main.plot("models", "B", "A", False )

NameError: name 'exit' is not defined